# 📊 UPI Transaction Analysis & Visualization

> **Author:** Your Name  
> **Dataset:** Synthetic UPI Transactions — India 2024 (1,000 records)  
> **Tools:** Python · Pandas · Matplotlib · Seaborn  
> **Focus Areas:** Spending Patterns · Category Analysis · Time-based Trends · Fraud Detection

---

### 📌 Project Structure
```
upi-transaction-analysis/
├── notebooks/
│   └── UPI_Transaction_Analysis.ipynb   ← You are here
├── data/
│   ├── generate_data.py
│   └── upi_transactions.csv
├── src/
│   ├── analysis.py
│   └── visualizations.py
├── outputs/                             ← All charts saved here
├── requirements.txt
└── README.md
```

---

## 🔧 Step 0: Environment Setup (Google Colab)

In [ ]:
# ── Install dependencies ───────────────────────────────────────────────────
!pip install pandas numpy matplotlib seaborn -q

# ── Clone repo (if running from GitHub) ───────────────────────────────────
# !git clone https://github.com/YOUR_USERNAME/upi-transaction-analysis.git
# %cd upi-transaction-analysis

# ── OR: Upload files manually to Colab and run this ───────────────────────
import sys, os
sys.path.append('.')
os.makedirs('data', exist_ok=True)
os.makedirs('outputs', exist_ok=True)
os.makedirs('src', exist_ok=True)

print('✅ Environment ready!')

## 📁 Step 1: Generate / Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ── Inline dataset generator (no file dependency) ──────────────────────────
np.random.seed(42)

CATEGORIES = ['Food & Dining','Shopping','Transport','Utilities',
              'Entertainment','Healthcare','Education','Travel']
MERCHANTS = {
    'Food & Dining':  ['Swiggy','Zomato',"McDonald's",'Domino\'s','Cafe Coffee Day','Blinkit','BigBasket'],
    'Shopping':       ['Amazon','Flipkart','Myntra','Meesho','Nykaa','Ajio','Snapdeal'],
    'Transport':      ['Ola','Uber','Rapido','Metro Card','IRCTC','RedBus','InDrive'],
    'Utilities':      ['BESCOM','Airtel','Jio','BWSSB','Gas Agency','Vi','BSNL'],
    'Entertainment':  ['Netflix','Hotstar','BookMyShow','Spotify','YouTube Premium','ZEE5','SonyLIV'],
    'Healthcare':     ['PharmEasy','1mg','Apollo Pharmacy','Medlife','Netmeds','Practo'],
    'Education':      ["Coursera",'Udemy',"BYJU'S",'Unacademy','WhiteHat Jr','Vedantu'],
    'Travel':         ['MakeMyTrip','Goibibo','OYO','Airbnb','Cleartrip','EaseMyTrip'],
}
STATES = ['Maharashtra','Karnataka','Tamil Nadu','Delhi','Telangana',
          'Gujarat','Rajasthan','Uttar Pradesh','West Bengal','Madhya Pradesh']
BANKS  = ['SBI','HDFC','ICICI','Axis','Kotak','PNB','Canara','BOB']
MODES  = ['UPI ID','QR Code','Phone Number']

n = 1000
dates = pd.date_range('2024-01-01','2024-12-31', periods=n)
dates = dates + pd.to_timedelta(np.random.randint(0, 86400, n), unit='s')
dates = dates.sort_values()

cats  = np.random.choice(CATEGORIES, n, p=[0.22,0.20,0.15,0.12,0.10,0.08,0.07,0.06])
merch = [np.random.choice(MERCHANTS[c]) for c in cats]

amount_mu = {'Food & Dining':300,'Shopping':1200,'Transport':180,'Utilities':600,
             'Entertainment':350,'Healthcare':500,'Education':2000,'Travel':3500}
amount_sd = {'Food & Dining':150,'Shopping':800,'Transport':100,'Utilities':300,
             'Entertainment':150,'Healthcare':400,'Education':1500,'Travel':2500}
amounts = [int(max(50, np.random.normal(amount_mu[c], amount_sd[c]))) for c in cats]

hours = dates.hour
is_fraud = np.array([
    (amt > 5000 and hr in range(1,5)) or
    (amt > 7000 and np.random.rand() < 0.4) or
    (amt > 4000 and hr in range(2,4) and np.random.rand() < 0.3)
    for amt, hr in zip(amounts, hours)
])

df = pd.DataFrame({
    'transaction_id': [f'TXN{str(i+1).zfill(5)}' for i in range(n)],
    'date':       dates.date,
    'datetime':   dates,
    'month':      dates.month,
    'month_name': dates.strftime('%B'),
    'day_of_week':dates.day_name(),
    'hour':       hours,
    'category':   cats,
    'merchant':   merch,
    'amount':     amounts,
    'payment_mode': np.random.choice(MODES, n, p=[0.50,0.30,0.20]),
    'sender_bank':  np.random.choice(BANKS, n),
    'receiver_bank':np.random.choice(BANKS, n),
    'state':        np.random.choice(STATES, n),
    'status':       np.where(is_fraud, 'Flagged', 'Success'),
    'is_fraud':     is_fraud.astype(int),
})

# Anomaly score
df['anomaly_score'] = (
    df['amount'].rank(pct=True) * 40 +
    df['hour'].apply(lambda h: 30 if h in range(1,5) else 0) +
    (df['amount'] > df['amount'].quantile(0.95)).astype(int) * 20
).clip(0, 100).round(1)

# Save
df.to_csv('data/upi_transactions.csv', index=False)
print(f'✅ Dataset: {len(df)} transactions | {df["is_fraud"].sum()} flagged ({df["is_fraud"].mean()*100:.1f}%)')
df.head()

---
## 🔍 Step 2: Exploratory Data Analysis (EDA)

In [ ]:
print('=' * 55)
print('  UPI TRANSACTION DATASET — SUMMARY')
print('=' * 55)
print(f"  Transactions     : {len(df):,}")
print(f"  Date Range       : {df['datetime'].min().date()} → {df['datetime'].max().date()}")
print(f"  Total Spend      : ₹{df['amount'].sum():,.0f}  (₹{df['amount'].sum()/100000:.2f} Lakhs)")
print(f"  Average Txn      : ₹{df['amount'].mean():,.0f}")
print(f"  Median Txn       : ₹{df['amount'].median():,.0f}")
print(f"  Max Single Txn   : ₹{df['amount'].max():,}")
print(f"  Unique Merchants : {df['merchant'].nunique()}")
print(f"  Categories       : {df['category'].nunique()}")
print(f"  Flagged Txns     : {df['is_fraud'].sum()} ({df['is_fraud'].mean()*100:.2f}%)")
print('=' * 55)

print('\nData Types & Nulls:')
print(df.dtypes)
print(f'\nNull values: {df.isnull().sum().sum()}')

In [ ]:
print('Statistical Summary:')
df[['amount','hour','anomaly_score']].describe().round(2)


## 🎨 Step 3: Plot Style Setup

In [ ]:
# ── Global style configuration ─────────────────────────────────────────────
DARK_BG  = '#0d1117'
CARD_BG  = '#161b22'
GRID_CLR = '#21262d'
TEXT_CLR = '#e6edf3'
MUTED    = '#8b949e'
ACCENT   = '#58a6ff'
ACCENT2  = '#3fb950'
DANGER   = '#f85149'
WARN     = '#d29922'
PALETTE  = ['#58a6ff','#3fb950','#d29922','#f85149','#bc8cff','#79c0ff','#56d364','#ffa657']

plt.rcParams.update({
    'figure.facecolor': DARK_BG, 'axes.facecolor': CARD_BG,
    'axes.edgecolor': GRID_CLR, 'axes.labelcolor': TEXT_CLR,
    'axes.titlecolor': TEXT_CLR, 'axes.titlesize': 13,
    'axes.titleweight': 'bold', 'axes.titlepad': 14,
    'axes.grid': True, 'grid.color': GRID_CLR, 'grid.linewidth': 0.6,
    'xtick.color': MUTED, 'ytick.color': MUTED,
    'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.facecolor': CARD_BG, 'legend.edgecolor': GRID_CLR,
    'legend.labelcolor': TEXT_CLR, 'legend.fontsize': 9,
    'text.color': TEXT_CLR, 'font.family': 'monospace',
})

def save_plot(fig, name):
    os.makedirs('outputs', exist_ok=True)
    path = f'outputs/{name}'
    fig.savefig(path, dpi=150, bbox_inches='tight', facecolor=DARK_BG)
    print(f'  ✅ Saved → {path}')

print('✅ Plot style configured.')

---
## 📊 Plot 1: KPI Dashboard Banner

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(18, 3))
fig.suptitle('UPI TRANSACTION ANALYSIS  ·  INDIA 2024', fontsize=14, fontweight='bold', color=TEXT_CLR, y=1.04)

kpis = [
    ('Total Transactions', f"{len(df):,}",                              ACCENT),
    ('Total Spend',        f"₹{df['amount'].sum()/100000:.2f}L",        ACCENT2),
    ('Avg Transaction',    f"₹{df['amount'].mean():,.0f}",              WARN),
    ('Fraud Rate',         f"{df['is_fraud'].mean()*100:.1f}%",         DANGER),
    ('Flagged Txns',       f"{df['is_fraud'].sum()}",                   DANGER),
]
for ax, (label, val, color) in zip(axes, kpis):
    ax.set_facecolor(CARD_BG)
    for s in ax.spines.values(): s.set_edgecolor(color + '66')
    ax.text(0.5, 0.62, val,   transform=ax.transAxes, ha='center', va='center', fontsize=24, fontweight='bold', color=color)
    ax.text(0.5, 0.22, label, transform=ax.transAxes, ha='center', va='center', fontsize=9,  color=MUTED)
    ax.set_xticks([]); ax.set_yticks([])

plt.tight_layout()
save_plot(fig, '01_kpi_banner.png')
plt.show()

---
## 📈 Plot 2: Monthly Spending Trend

In [ ]:
month_order = ['January','February','March','April','May','June',
               'July','August','September','October','November','December']
monthly = df.groupby('month_name').agg(
    total_spend=('amount','sum'),
    count=('amount','count'),
    avg=('amount','mean'),
    flagged=('is_fraud','sum')
).reindex(month_order).reset_index()

fig, ax = plt.subplots(figsize=(14, 5))
x = range(len(monthly))
y = monthly['total_spend'] / 1000

ax.fill_between(x, y, alpha=0.18, color=ACCENT)
ax.plot(x, y, color=ACCENT, linewidth=2.5, marker='o', markersize=7,
        markerfacecolor=DARK_BG, markeredgecolor=ACCENT, markeredgewidth=2, label='Monthly Spend')

# Annotate max month
idx = y.idxmax()
ax.annotate(f"Peak: ₹{y[idx]:.1f}k", xy=(idx, y[idx]),
            xytext=(idx+0.3, y[idx]+3), color=ACCENT2, fontsize=9, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color=ACCENT2, lw=1.5))

ax.set_xticks(list(x))
ax.set_xticklabels([m[:3] for m in monthly['month_name']], fontsize=9)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f'₹{v:.0f}k'))
ax.set_title('Monthly UPI Spending Trend — 2024')
ax.set_xlabel('Month'); ax.set_ylabel('Total Spend (₹ thousands)')
ax.legend()
plt.tight_layout()
save_plot(fig, '02_monthly_trend.png')
plt.show()

print('\nMonthly Summary Table:')
monthly[['month_name','count','total_spend','avg','flagged']].rename(
    columns={'month_name':'Month','count':'Txns','total_spend':'Total Spend','avg':'Avg Txn','flagged':'Flagged'}
).style.format({'Total Spend': '₹{:,.0f}', 'Avg Txn': '₹{:,.0f}', 'Flagged': '{:.0f}'})

---
## 🥧 Plot 3: Category Breakdown

In [ ]:
cat_df = df.groupby('category').agg(
    total_spend=('amount','sum'),
    count=('amount','count'),
    avg=('amount','mean'),
    flagged=('is_fraud','sum')
).sort_values('total_spend', ascending=False).reset_index()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Donut chart
wedges, texts, autotexts = ax1.pie(
    cat_df['total_spend'], labels=None,
    colors=PALETTE[:len(cat_df)],
    autopct='%1.1f%%', pctdistance=0.78,
    wedgeprops=dict(width=0.55, edgecolor=DARK_BG, linewidth=2.5),
    startangle=90
)
for at in autotexts: at.set(fontsize=8, color=TEXT_CLR, fontweight='bold')
ax1.legend(cat_df['category'], loc='lower left', fontsize=8, framealpha=0.3, ncol=2)
ax1.set_title('Spend Share by Category')

# Horizontal bar
bars = ax2.barh(cat_df['category'], cat_df['total_spend']/1000,
                color=PALETTE[:len(cat_df)], edgecolor=DARK_BG, height=0.65)
for bar, val in zip(bars, cat_df['total_spend']/1000):
    ax2.text(val + 0.5, bar.get_y() + bar.get_height()/2,
             f'₹{val:.1f}k', va='center', fontsize=9, color=TEXT_CLR)
ax2.set_title('Total Spend per Category')
ax2.set_xlabel('Total Spend (₹ thousands)')
ax2.invert_yaxis()

plt.tight_layout()
save_plot(fig, '03_category_breakdown.png')
plt.show()

---
## 🏪 Plot 4: Top Merchants by Spend

In [ ]:
merchant_df = df.groupby('merchant').agg(
    total_spend=('amount','sum'), count=('amount','count')
).sort_values('total_spend', ascending=False).head(10).reset_index()

fig, ax = plt.subplots(figsize=(12, 6))
colors = [PALETTE[i % len(PALETTE)] for i in range(len(merchant_df))]
bars = ax.barh(merchant_df['merchant'], merchant_df['total_spend']/1000,
               color=colors, edgecolor=DARK_BG, height=0.6)
for bar, val, cnt in zip(bars, merchant_df['total_spend']/1000, merchant_df['count']):
    ax.text(val + 0.3, bar.get_y() + bar.get_height()/2,
            f'₹{val:.1f}k  ({cnt} txns)', va='center', fontsize=9, color=TEXT_CLR)
ax.set_title('Top 10 Merchants by Total UPI Spend')
ax.set_xlabel('Total Spend (₹ thousands)')
ax.invert_yaxis()
plt.tight_layout()
save_plot(fig, '04_top_merchants.png')
plt.show()

---
## 📅 Plot 5: Day of Week & Hourly Patterns

In [ ]:
dow_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
dow_df = df.groupby('day_of_week').agg(
    total_spend=('amount','sum'), count=('amount','count'), avg=('amount','mean')
).reindex(dow_order).reset_index()

hourly_df = df.groupby('hour').agg(
    count=('amount','count'), flagged=('is_fraud','sum'), total=('amount','sum')
).reset_index()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Spend by day
colors = [ACCENT2 if d in ['Saturday','Sunday'] else ACCENT for d in dow_df['day_of_week']]
axes[0,0].bar(dow_df['day_of_week'].str[:3], dow_df['total_spend']/1000, color=colors, edgecolor=DARK_BG, width=0.6)
axes[0,0].set_title('Total Spend by Day of Week')
axes[0,0].set_ylabel('Spend (₹k)')
axes[0,0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda v,_: f'₹{v:.0f}k'))

# Txn count by day
axes[0,1].bar(dow_df['day_of_week'].str[:3], dow_df['count'], color=colors, edgecolor=DARK_BG, width=0.6)
axes[0,1].set_title('Transactions by Day of Week')
axes[0,1].set_ylabel('Count')

# Hourly transaction count
axes[1,0].fill_between(hourly_df['hour'], hourly_df['count'], alpha=0.2, color=ACCENT)
axes[1,0].plot(hourly_df['hour'], hourly_df['count'], color=ACCENT, linewidth=2, marker='o', markersize=4, markerfacecolor=DARK_BG)
axes[1,0].set_title('Transaction Volume by Hour of Day')
axes[1,0].set_xlabel('Hour'); axes[1,0].set_ylabel('Count')
axes[1,0].set_xticks(range(0, 24, 2))

# Hourly spend
axes[1,1].fill_between(hourly_df['hour'], hourly_df['total']/1000, alpha=0.2, color=ACCENT2)
axes[1,1].plot(hourly_df['hour'], hourly_df['total']/1000, color=ACCENT2, linewidth=2, marker='o', markersize=4, markerfacecolor=DARK_BG)
axes[1,1].set_title('Total Spend by Hour of Day')
axes[1,1].set_xlabel('Hour'); axes[1,1].set_ylabel('Spend (₹k)')
axes[1,1].set_xticks(range(0, 24, 2))

plt.tight_layout()
save_plot(fig, '05_time_patterns.png')
plt.show()

---
## 🔥 Plot 6: Activity Heatmap (Day × Hour)

In [ ]:
pivot = df.groupby(['day_of_week','hour'])['amount'].count().unstack(fill_value=0)
pivot = pivot.reindex([d for d in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'] if d in pivot.index])

fig, ax = plt.subplots(figsize=(18, 5))
cmap = sns.color_palette('Blues', as_cmap=True)
sns.heatmap(pivot, ax=ax, cmap='YlOrRd', linewidths=0.3, linecolor=DARK_BG,
            annot=True, fmt='d', annot_kws={'size': 8, 'color': TEXT_CLR},
            cbar_kws={'label': 'Transaction Count'})
ax.set_title('Transaction Activity Heatmap  (Day of Week × Hour of Day)', fontsize=14)
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Day of Week')
plt.tight_layout()
save_plot(fig, '06_activity_heatmap.png')
plt.show()

---
## 🚨 Plot 7: Fraud Detection Analysis

In [ ]:
fraud_df    = df[df['is_fraud'] == 1]
normal_df   = df[df['is_fraud'] == 0]

fig = plt.figure(figsize=(18, 10))
fig.suptitle('FRAUD DETECTION ANALYSIS', fontsize=15, fontweight='bold', color=DANGER, y=1.01)
gs  = fig.add_gridspec(2, 3, hspace=0.45, wspace=0.4)

# (A) Status split pie
ax1 = fig.add_subplot(gs[0, 0])
counts = df['status'].value_counts()
ax1.pie(counts, labels=counts.index, colors=[ACCENT2, DANGER],
        autopct='%1.1f%%', wedgeprops=dict(edgecolor=DARK_BG, linewidth=2.5),
        startangle=90, pctdistance=0.78)
ax1.set_title('Transaction Status Split')

# (B) Flagged by hour
ax2 = fig.add_subplot(gs[0, 1:])
ax2.bar(hourly_df['hour'], hourly_df['count'], label='Normal',  color=ACCENT,  alpha=0.5, width=0.8)
ax2.bar(hourly_df['hour'], hourly_df['flagged'], label='Flagged', color=DANGER, width=0.8)
ax2.set_title('Flagged Transactions by Hour of Day')
ax2.set_xlabel('Hour of Day'); ax2.set_ylabel('Count')
ax2.legend(); ax2.set_xticks(range(0, 24, 2))
ax2.axvspan(1, 5, alpha=0.08, color=DANGER, label='High-risk window')

# (C) Fraud by month
ax3 = fig.add_subplot(gs[1, :2])
x = range(len(monthly))
w = 0.35
ax3.bar([i - w/2 for i in x], monthly['count'],   width=w, label='Total',   color=ACCENT,  alpha=0.8)
ax3.bar([i + w/2 for i in x], monthly['flagged'], width=w, label='Flagged', color=DANGER)
ax3.set_xticks(list(x))
ax3.set_xticklabels([m[:3] for m in monthly['month_name']], fontsize=8)
ax3.set_title('Total vs Flagged Transactions by Month')
ax3.set_ylabel('Count'); ax3.legend()

# (D) Amount distribution
ax4 = fig.add_subplot(gs[1, 2])
ax4.hist(normal_df['amount'], bins=35, alpha=0.6, color=ACCENT,  label='Normal',  density=True)
ax4.hist(fraud_df['amount'],  bins=20, alpha=0.8, color=DANGER, label='Flagged', density=True)
ax4.axvline(df['amount'].quantile(0.95), color=WARN, linestyle='--', linewidth=1.5, label='95th pct')
ax4.set_title('Amount Distribution: Fraud vs Normal')
ax4.set_xlabel('Transaction Amount (₹)'); ax4.set_ylabel('Density')
ax4.legend()

plt.tight_layout()
save_plot(fig, '07_fraud_analysis.png')
plt.show()

print(f'\n🚨 Fraud Insights:')
print(f'   Total Flagged     : {len(fraud_df)} transactions')
print(f'   Fraud Rate        : {len(fraud_df)/len(df)*100:.2f}%')
print(f'   Avg Fraud Amount  : ₹{fraud_df["amount"].mean():,.0f}')
print(f'   Avg Normal Amount : ₹{normal_df["amount"].mean():,.0f}')
print(f'   Peak Fraud Hour   : {fraud_df["hour"].mode()[0]}:00')

---
## 🎯 Plot 8: Anomaly Score Analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Distribution
ax1.hist(normal_df['anomaly_score'], bins=30, alpha=0.65, color=ACCENT,  label='Normal',  density=True)
ax1.hist(fraud_df['anomaly_score'],  bins=15, alpha=0.85, color=DANGER, label='Flagged', density=True)
ax1.axvline(70, color=WARN, linestyle='--', linewidth=2, label='Risk Threshold (70)')
ax1.set_title('Anomaly Score Distribution')
ax1.set_xlabel('Anomaly Score (0–100)'); ax1.set_ylabel('Density')
ax1.legend()

# Top 20 high-risk
top_risky = df.nlargest(20, 'anomaly_score')
colors_r = [DANGER if f else ACCENT for f in top_risky['is_fraud']]
labels_r = [f"{m} ({t})" for m, t in zip(top_risky['merchant'], top_risky['transaction_id'])]
ax2.barh(labels_r, top_risky['anomaly_score'], color=colors_r, edgecolor=DARK_BG, height=0.7)
ax2.axvline(70, color=WARN, linestyle='--', linewidth=1.5)
ax2.set_title('Top 20 High-Risk Transactions')
ax2.set_xlabel('Anomaly Score'); ax2.invert_yaxis()
ax2.tick_params(axis='y', labelsize=7.5)

plt.tight_layout()
save_plot(fig, '08_anomaly_scores.png')
plt.show()

# High risk table
print('\nTop 10 Highest Anomaly Score Transactions:')
df.nlargest(10, 'anomaly_score')[['transaction_id','merchant','category','amount','hour','anomaly_score','status']].reset_index(drop=True)

---
## 💳 Plot 9: Payment Mode & Bank Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Payment mode count
mode_df = df.groupby('payment_mode')['amount'].agg(['count','sum']).reset_index()
axes[0].bar(mode_df['payment_mode'], mode_df['count'], color=PALETTE[:3], edgecolor=DARK_BG, width=0.5)
axes[0].set_title('Transactions by Payment Mode')
axes[0].set_ylabel('Count')
for bar, val in zip(axes[0].patches, mode_df['count']):
    axes[0].text(bar.get_x()+bar.get_width()/2, bar.get_height()+3, str(val), ha='center', fontsize=9)

# Payment mode spend
axes[1].bar(mode_df['payment_mode'], mode_df['sum']/1000, color=PALETTE[3:6], edgecolor=DARK_BG, width=0.5)
axes[1].set_title('Total Spend by Payment Mode')
axes[1].set_ylabel('Total Spend (₹k)')

# Bank spend
bank_df = df.groupby('sender_bank')['amount'].sum().sort_values(ascending=False).reset_index()
colors_b = [PALETTE[i % len(PALETTE)] for i in range(len(bank_df))]
axes[2].bar(bank_df['sender_bank'], bank_df['amount']/1000, color=colors_b, edgecolor=DARK_BG, width=0.6)
axes[2].set_title('Total Spend by Sender Bank')
axes[2].set_ylabel('Spend (₹k)')
axes[2].tick_params(axis='x', rotation=30)

plt.tight_layout()
save_plot(fig, '09_payment_bank.png')
plt.show()

---
## 🗺️ Plot 10: State-wise Spending

In [ ]:
state_df = df.groupby('state').agg(
    total_spend=('amount','sum'), count=('amount','count'), fraud=('is_fraud','sum')
).sort_values('total_spend', ascending=False).reset_index()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Spend bar
colors_s = [PALETTE[i % len(PALETTE)] for i in range(len(state_df))]
bars = ax1.bar(state_df['state'], state_df['total_spend']/1000, color=colors_s, edgecolor=DARK_BG, width=0.65)
for bar, val in zip(bars, state_df['total_spend']/1000):
    ax1.text(bar.get_x()+bar.get_width()/2, bar.get_height()+1, f'₹{val:.0f}k',
             ha='center', va='bottom', fontsize=8, color=TEXT_CLR)
ax1.set_title('Total UPI Spend by State')
ax1.set_ylabel('Total Spend (₹ thousands)')
ax1.tick_params(axis='x', rotation=35)

# Fraud by state
ax2.bar(state_df['state'], state_df['fraud'], color=DANGER, edgecolor=DARK_BG, width=0.65, alpha=0.85)
ax2.set_title('Flagged Transactions by State')
ax2.set_ylabel('Flagged Count')
ax2.tick_params(axis='x', rotation=35)

plt.tight_layout()
save_plot(fig, '10_state_analysis.png')
plt.show()

---
## 🔗 Plot 11: Correlation & Amount Distribution

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Overall amount distribution
axes[0].hist(df['amount'], bins=40, color=ACCENT, edgecolor=DARK_BG, alpha=0.8)
axes[0].axvline(df['amount'].mean(),   color=ACCENT2, linestyle='--', linewidth=2, label=f"Mean ₹{df['amount'].mean():,.0f}")
axes[0].axvline(df['amount'].median(), color=WARN,    linestyle='--', linewidth=2, label=f"Median ₹{df['amount'].median():,.0f}")
axes[0].set_title('Transaction Amount Distribution')
axes[0].set_xlabel('Amount (₹)'); axes[0].set_ylabel('Frequency')
axes[0].legend(fontsize=8)

# Boxplot by category
cat_order = df.groupby('category')['amount'].median().sort_values(ascending=False).index
cat_data  = [df[df['category']==c]['amount'].values for c in cat_order]
bp = axes[1].boxplot(cat_data, patch_artist=True, medianprops=dict(color=WARN, linewidth=2))
for patch, color in zip(bp['boxes'], PALETTE):
    patch.set_facecolor(color + '55'); patch.set_edgecolor(color)
axes[1].set_xticks(range(1, len(cat_order)+1))
axes[1].set_xticklabels([c.split(' ')[0] for c in cat_order], fontsize=8, rotation=30)
axes[1].set_title('Amount Distribution by Category (Boxplot)')
axes[1].set_ylabel('Amount (₹)')

# Correlation heatmap
num_cols = ['amount','hour','month','is_fraud','anomaly_score']
corr = df[num_cols].corr()
sns.heatmap(corr, ax=axes[2], annot=True, fmt='.2f', cmap='RdYlGn', center=0,
            linewidths=0.5, linecolor=DARK_BG, annot_kws={'size': 10},
            cbar_kws={'label': 'Correlation'})
axes[2].set_title('Feature Correlation Matrix')

plt.tight_layout()
save_plot(fig, '11_distributions_correlations.png')
plt.show()

---
## 📋 Step 4: Key Findings & Insights

In [ ]:
top_cat    = cat_df.iloc[0]['category']
top_merch  = merchant_df.iloc[0]['merchant']
top_state  = state_df.iloc[0]['state']
peak_hour  = hourly_df.loc[hourly_df['count'].idxmax(), 'hour']
peak_day   = dow_df.loc[dow_df['total_spend'].idxmax(), 'day_of_week']
peak_month = monthly.loc[monthly['total_spend'].idxmax(), 'month_name']

print('=' * 60)
print('  📊 UPI TRANSACTION ANALYSIS — KEY FINDINGS')
print('=' * 60)
print(f"\n  📈 Spending Patterns")
print(f"  • Total spend       : ₹{df['amount'].sum():,.0f}  ({df['amount'].sum()/100000:.2f} Lakhs)")
print(f"  • Peak month        : {peak_month}")
print(f"  • Peak day          : {peak_day}")
print(f"  • Peak hour         : {peak_hour}:00")
print(f"\n  🏷️  Category Analysis")
print(f"  • Top category      : {top_cat}  (₹{cat_df.iloc[0]['total_spend']:,.0f})")
print(f"  • Top merchant      : {top_merch}")
print(f"  • Top state         : {top_state}")
print(f"\n  🚨 Fraud Detection")
print(f"  • Flagged txns      : {df['is_fraud'].sum()} ({df['is_fraud'].mean()*100:.2f}%)")
print(f"  • Avg fraud amount  : ₹{fraud_df['amount'].mean():,.0f}")
print(f"  • High-risk window  : 01:00 – 05:00 AM")
print(f"  • Fraud threshold   : ₹5,000+ in odd hours")
print(f"\n  🎯 Anomaly Detection")
print(f"  • High-risk (>70)   : {len(df[df['anomaly_score'] > 70])} transactions")
print(f"  • Max anomaly score : {df['anomaly_score'].max()}")
print('=' * 60)

---
## 💾 Step 5: Export Results

In [ ]:
import os
os.makedirs('outputs', exist_ok=True)

# Export summary tables
monthly.to_csv('outputs/monthly_summary.csv', index=False)
cat_df.to_csv('outputs/category_summary.csv', index=False)
merchant_df.to_csv('outputs/top_merchants.csv', index=False)
fraud_df.to_csv('outputs/flagged_transactions.csv', index=False)
df.nlargest(50, 'anomaly_score').to_csv('outputs/high_risk_transactions.csv', index=False)

# List all outputs
print('✅ Exported files:')
for f in sorted(os.listdir('outputs')):
    size = os.path.getsize(f'outputs/{f}')
    print(f'   {f:<45} {size/1024:.1f} KB')

---

## ✅ Project Complete!

All 11 visualizations have been generated and saved in the `outputs/` folder.

### 📤 Push to GitHub
```bash
git init
git add .
git commit -m "feat: UPI Transaction Analysis - complete EDA and fraud detection"
git branch -M main
git remote add origin https://github.com/YOUR_USERNAME/upi-transaction-analysis.git
git push -u origin main
```

---
*Made with Python · Pandas · Matplotlib · Seaborn*